## Tic-Tac-Toe Agent
​
In this notebook, you will learn to build an RL agent (using Q-learning) that learns to play Numerical Tic-Tac-Toe with odd numbers. The environment is playing randomly with the agent, i.e. its strategy is to put an even number randomly in an empty cell. The following is the layout of the notebook:
        - Defining epsilon-greedy strategy
        - Tracking state-action pairs for convergence
        - Define hyperparameters for the Q-learning algorithm
        - Generating episode and applying Q-update equation
        - Checking convergence in Q-values

#### Importing libraries
Write the code to import Tic-Tac-Toe class from the environment file

In [ ]:

from TCGame_Env import TicTacToe#- import your class from environment file
import collections
import numpy as np
import random
import pickle
import time
from matplotlib import pyplot as plt

In [ ]:
# Function to convert state array into a string to store it as keys in the dictionary
# states in Q-dictionary will be of form: x-4-5-3-8-x-x-x-x
#   x | 4 | 5
#   ----------
#   3 | 8 | x
#   ----------
#   x | x | x

def Q_state(state):

    return ('-'.join(str(e) for e in state)).replace('nan','x')

In [ ]:
#read the pickle files and laod the Remards and Q-values for the states fromt he file
#Incase of no file existing , the function will initialize the rewards , Q_dict and States_track dictionaries and return the same.
#It also returns a tuple with the boolean value indicating the initialization
import os

def read_pickle():
    policy_flag = False
    rewards_flag = False
    states_flag = False
    
    Q_dict = collections.defaultdict(dict)

    States_track = collections.defaultdict(dict)

    rewards_tracked = {(1,np.nan,np.nan,np.nan,6,np.nan,np.nan,np.nan,np.nan):[],
                       (np.nan,np.nan,2,np.nan,7,np.nan,np.nan,np.nan,np.nan):[],
                       (np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan):[],
                       (np.nan,np.nan,7,3,2,np.nan,np.nan,8,np.nan):[],
                       (np.nan,8,np.nan,6,1,3,np.nan,np.nan,np.nan):[],
                       (np.nan,8,5,np.nan,9,np.nan,np.nan,np.nan,6):[]}
    
    if os.path.isfile('./Policy.pkl'):
        with open('Policy.pkl', 'rb') as p_handle:
            Q_dict = pickle.load(p_handle)
            policy_flag = True
            
    if os.path.isfile('./Rewards.pkl'):
        with open('Rewards.pkl', 'rb') as r_handle:
            rewards_tracked = pickle.load(r_handle)    
            rewards_flag = True
    
    if os.path.isfile('./States_tracked.pkl'):
        with open('States_tracked.pkl', 'rb') as s_handle:
            States_track = pickle.load(s_handle)    
            states_flag = True

    if policy_flag and rewards_flag and states_flag:
        return (True ,Q_dict,rewards_tracked,States_track)
    else:
        return (False,Q_dict,rewards_tracked,States_track)

In [ ]:
# Defining a function which will return valid (all possible actions) actions corresponding to a state
# Important to avoid errors during deployment.

def valid_actions(state):

    valid_Actions = []
    
    valid_Actions = [i for i in env.action_space(state)[0]] ###### -------please call your environment as env
    return valid_Actions

In [ ]:
# Defining a function which will add new Q-values to the Q-dictionary. 
def add_to_dict(state):
    state1 = Q_state(state)
    
    valid_act = valid_actions(state)
    if state1 not in Q_dict.keys():
        for action in valid_act:
            Q_dict[state1][action]=0

#### Epsilon-greedy strategy - Write your code here

(you can build your epsilon-decay function similar to the one given at the end of the notebook)

In [ ]:
# Defining epsilon-greedy policy. You can choose any function epsilon-decay strategy
def greedy_epsilon(state, time):
    max_epsilon = 1.0
    min_epsilon = 0.001
    #time = np.arange(0,99)
    z = np.random.random()
    
    epsilon =  min_epsilon + (max_epsilon - min_epsilon) * np.exp(-0.00005*time)
    if z > epsilon:
        action = max(Q_dict[Q_state(state)],key=Q_dict[Q_state(state)].get)   #Exploitation: this gets the action corresponding to max q-value of current state
    else:
        action = np.random.choice(np.arange(0,len (Q_dict[Q_state(state)])))    #Exploration: randomly choosing and action

        lst = list(Q_dict[Q_state(state)].items())
        action = lst[action]
       
        pos = str(action).split(",")[0][2:]
        val = str(action).split(",")[1][:-1]
        action = [int(pos) ,int(val)]
    return action



#### Tracking the state-action pairs for checking convergence - write your code here

In [ ]:
#Read the pickle files and load the dictionaries based on what is loaded / initialized
initialize = read_pickle()

Q_dict = initialize[1]
States_track = initialize[3]
rewards_tracked = initialize[2]

print(len(Q_dict))
print(len(rewards_tracked))
print(len(States_track))

In [ ]:
# Initialise states to be tracked
def initialise_tracking_states():
    sample_q_values = [((1,np.nan,np.nan,np.nan,6,np.nan,np.nan,np.nan,np.nan),(1,3)),
                       ((np.nan,np.nan,2,np.nan,7,np.nan,np.nan,np.nan,np.nan),(8,3)),
                       ((np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan),(5,7)),
                       ((np.nan,np.nan,7,3,2,np.nan,np.nan,8,np.nan),(6,5)),
                       ((np.nan,8,np.nan,6,1,3,np.nan,np.nan,np.nan),(6,7)),
                       ((np.nan,8,5,np.nan,9,np.nan,np.nan,np.nan,6),(6,1))
                      ]    #select any 4 Q-values
    for q_values in sample_q_values:
        state = q_values[0]
        action = q_values[1]
        States_track[state][action] = []    #this is an array which will have appended values of that state-action pair for every 2000th episode           
        

In [ ]:
#Defining a function to save the Q-dictionary as a pickle file

def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
def save_tracking_states():
    for state in States_track.keys():
        for action in States_track[state].keys():
            if Q_state(state) in Q_dict and action in Q_dict[ Q_state(state)]:
                   States_track[state][action].append(Q_dict[Q_state(state)][action])

In [ ]:
#only needed if the values are not loaded from the pickle files.
if not initialize[0]:
    initialise_tracking_states()

#### Define hyperparameters  ---write your code here

In [ ]:
#Defining parameters for the experiment

EPISODES = 100000
STEPS = 5
LR = 0.01                   #learning rate
GAMMA = 0.9

threshold = 1000       #every these many episodes, the 4 Q-values will be stored/appended (convergence graphs)
policy_threshold = 3000

### Q-update loop ---write your code here

In [ ]:
import copy
start_time = time.time()
#############################################################################
for episode in range(EPISODES):
    ##### Start writing your code from the next line
    if (episode % 100000 == 0):
        print ('Episode -' + str(episode))
    env = TicTacToe()
    
    initial_state = env.state 
    curr_state = env.state
    
    add_to_dict(curr_state)
    
    time_step = 0
    reward = None

    total_reward = 0
    
    while time_step < STEPS:
       
        curr_action = greedy_epsilon(curr_state, time_step)

        state_reward_term = env.step(copy.deepcopy(curr_state), curr_action)
        
        next_state = state_reward_term[0]
        reward = state_reward_term[1]
        terminal= state_reward_term[2]
        
        tup_curr_action = tuple(curr_action)
        if not terminal:
            
            add_to_dict(next_state)
            # For non terminal cases get the action with the action with the max value in the next state
            # Use the q-learning equation to assign the value for the current state
            try:
                max_next = max(Q_dict[Q_state(next_state)],key=Q_dict[Q_state(next_state)].get)   #this gets the action corresponding to max q-value of next state
                Q_dict[Q_state(curr_state)][tup_curr_action] += LR * ((reward + (GAMMA*(Q_dict[Q_state(next_state)][tuple(max_next)]))) - Q_dict[Q_state(curr_state)][tup_curr_action] ) 
            except:
                print(next_state)
                print (Q_dict[Q_state(next_state)])
                break
                
            total_reward += reward
            #TRACKING REWARDS
            if tuple(curr_state) in list(rewards_tracked):     #storing rewards
                rewards_tracked[tuple(curr_state)].append(total_reward)
                
            curr_state = next_state       #state(t) became state(t-1)
            
            time_step += 1
    
        else:
            #In case of terminal state, the state value for the next state is zero and hence the Q-learning equation has been changed a bit to reflect that. 
            Q_dict[Q_state(curr_state)][tup_curr_action] += LR * (reward - Q_dict[Q_state(curr_state)][tup_curr_action]) 
            total_reward += reward
            #TRACKING REWARDS
            if tuple(curr_state) in list(rewards_tracked):     #storing rewards
                rewards_tracked[tuple(curr_state)].append(total_reward)
            time_step += 1
            break

      
    if ((episode+1) % threshold) == 0:   
        save_tracking_states()
    
################################################################################    
elapsed_time = time.time() - start_time
save_obj(States_track,'States_tracked')   
save_obj(Q_dict,'Policy')
save_obj(rewards_tracked,'Rewards')

In [ ]:
elapsed_time

#### Check the Q-dictionary

In [ ]:
len(Q_dict)

#### Some Key observations -
- (1,np.nan,np.nan,np.nan,6,np.nan,np.nan,np.nan,np.nan) - Action (8,9) is the best action and (1,7) is the worst action.
- (np.nan,np.nan,2,np.nan,7,np.nan,np.nan,np.nan,np.nan) - Action (6,5) is the best action as suggested by the model.(5,5) is the worst action.
- (np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan) - Action (1,1) is the best action and (4,5) is the worst action. 
- (np.nan,np.nan,7,3,2,np.nan,np.nan,8,np.nan) - Action (1,5) has the highest value as per model and results in a agent win. (5,1) worst options
- (np.nan,8,np.nan,6,1,3,np.nan,np.nan,np.nan) - Action (8,5) has the highest value as per model. 
- (np.nan,8,5,np.nan,9,np.nan,np.nan,np.nan,6) - Action (6,1) has the highest value as per model and results in a agent win.(0,1) is the worst option

In [ ]:
# try checking for one of the states - that which action your agent thinks is the best  -----This will not be evaluated
initialize = read_pickle()   #Read the pickle files for rewards , qdict and state_track

Q_dict = initialize[1]
States_track = initialize[3]
rewards_tracked = initialize[2]

for state in States_track.keys():
    print (state)
    print (Q_dict[Q_state(state)])
    print ('-------------------------')


#### Check the states tracked for Q-values convergence
(non-evaluative)

In [ ]:

i = 0
j = 0

fig,axs = plt.subplots(3,2,figsize=(15,15))
for state in States_track.keys():
    for action in States_track[state].keys():
        axs[i,j].set_title('state='+str(state)+' action='+str(action))
        axs[i,j].plot (np.asarray(States_track[state][action]))
        if (i==0 and j == 0):
            j += 1
        elif (i==0 and j==1):
            i += 1
            j = 0
        elif (i==1 and j==0):
            j += 1
        elif (i==1 and j==1):
            i += 1
            j = 0
        else:
            j += 1                   
plt.show()       

### Epsilon - decay check

In [ ]:
max_epsilon = 1.0
min_epsilon = 0.001
time = np.arange(0,100000)
epsilon = []
for i in range(0,100000):
    epsilon.append(min_epsilon + (max_epsilon - min_epsilon) * np.exp(-0.00005*i))

In [ ]:
plt.plot(time, epsilon)
plt.show()